You have to work on the files:
*  [Books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz)
*  [Book ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz)
*  [Users](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz)
*  [Goodbooks books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz)
*  [Goodbooks ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz)

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    To read those files, you need to use the `encoding = 'latin-1'` option.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com), otherwise no  answer will be given.

In [1]:
import pandas as pd
import re
import numpy as np

In [13]:
books = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz', sep=";",low_memory=False, encoding="latin-1")
ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz', sep=";", encoding='latin-1')
users = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz', sep=";", encoding='latin-1')
good_books = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz', sep=",", encoding='latin-1', low_memory=False)
good_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz', sep=",", encoding='latin-1')

### 1. Normalize the location field of *Users* dataset, splitting into city, region, country.

In [14]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [18]:
country = []
city =[]
region = []
re_location = re.compile('(?P<City>.+),\s*(?P<Region>.+),\s*(?P<Country>.+)')
for line in users['Location']:
    occ = re_location.search(line)
    if occ:
        city.append(occ.group('City'))
        region.append(occ.group('Region'))
        country.append(occ.group('Country'))
    else:
        city.append('None')
        region.append('None')
        country.append('None')
        
users['City'] = city
users['Region'] = region
users['Country'] = country

users.drop(columns='Location', inplace=True)


In [19]:
users.head()

,User-ID,Age,City,Region,Country
0,1,NaN,nyc,new york,usa
1,2,18.0,stockton,california,usa
2,3,NaN,moscow,yukon territory,russia
3,4,17.0,porto,v.n.gaia,portugal
4,5,NaN,farnborough,hants,united kingdom


### 2. For each book in the *Books* dataset, compute its average rating.

In [30]:
ratings_by_isbn = ratings.groupby('ISBN', as_index=False)[['ISBN','Book-Rating']].mean()
ratings_by_isbn

,ISBN,Book-Rating
0,0330299891,3.0
1,0375404120,1.5
2,0586045007,0.0
3,9022906116,3.5
4,9032803328,0.0
...,...,...
340551,cn113107,0.0
340552,ooo7156103,7.0
340553,Â§423350229,0.0
340554,Â´3499128624,8.0


In [31]:
books = books.merge(ratings_by_isbn, on='ISBN', how='left')
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,0.000000
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,4.928571
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,5.000000
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,4.272727
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,0.000000


### 3. For each book in the *GoodBooks* dataset, compute its average rating.

In [32]:
good_books['average'] = (good_books['ratings_1'] * 1 + good_books['ratings_2'] * 2 + good_books['ratings_3'] * 3 + good_books['ratings_4'] * 4 + good_books['ratings_5'] * 5) / good_books['work_ratings_count']

In [33]:
good_books[['book_id','original_title','average']].sort_values('average', ascending=False)

,book_id,original_title,average
24,25,Harry Potter and the Deathly Hallows,4.612756
26,27,Harry Potter and the Half-Blood Prince,4.537725
17,18,Harry Potter and the Prisoner of Azkaban,4.527866
23,24,Harry Potter and the Goblet of Fire,4.525950
20,21,Harry Potter and the Order of the Phoenix,4.463372
...,...,...,...
33,34,Fifty Shades of Grey,3.665074
27,28,Lord of the Flies,3.642994
2,3,Twilight,3.571067
48,49,"New Moon (Twilight, #2)",3.521807


### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset `merged books`. The books that have not been merged together will not appear in `merged books`.

In [34]:
merged_books = books.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False, )['Book-Rating'].agg(['max','min','size','mean']).reset_index()
merged_books = merged_books[merged_books['size'] > 1]
merged_books

,Book-Title,Book-Author,Publisher,max,min,size,mean
41,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,6.000000,0.000000,2,3.000000
59,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,3.400000,0.000000,2,1.700000
255,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,7.250000,6.666667,2,6.958333
465,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,5.000000,0.000000,3,3.111111
713,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,0.000000,0.000000,2,0.000000
...,...,...,...,...,...,...,...
265525,Zia,Scott O'Dell,Laurel-Leaf Books,3.428571,1.666667,2,2.547619
265526,Zia,Scott O'Dell,Yearling Books,2.000000,0.000000,2,1.000000
265562,Zimmermann Telegram,Barbara Tuchman,Ballantine Books,4.000000,0.000000,2,2.000000
265625,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.000000,10.000000,4,10.000000


### 5. For each book in `merged books` compute its average rating.

The average is computed considering all books in `books` that have been merged.

In [35]:
merged_books['mean']

41         3.000000
59         1.700000
255        6.958333
465        3.111111
713        0.000000
            ...    
265525     2.547619
265526     1.000000
265562     2.000000
265625    10.000000
266117     3.833333
Name: mean, Length: 4725, dtype: float64

### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

In [36]:
merged_books[['max','min']]

,max,min
41,6.000000,0.000000
59,3.400000,0.000000
255,7.250000,6.666667
465,5.000000,0.000000
713,0.000000,0.000000
...,...,...
265525,3.428571,1.666667
265526,2.000000,0.000000
265562,4.000000,0.000000
265625,10.000000,10.000000


### 7. For each book in `goodbooks`, compute the list of its authors. Assuming that the number of reviews with a text (column `work_text_reviews_count`) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the *shared number of reviews with a text*.

In [37]:
good_books[['author1','author2','author3','author4','author5']] = good_books['authors'].str.split(',', expand=True)

In [38]:
good_books['authors_count'] = good_books[['author1','author2','author3','author4','author5']].notnull().sum(axis=1).tolist()

In [41]:
good_books['shared_number_of_reviews_with_a_text'] = round(good_books['work_text_reviews_count'] / good_books['authors_count'])

In [42]:
good_books_author1 = good_books[good_books['author1'].isnull() == False].drop(columns=['author2','author3', 'author4', 'author5'])
good_books_author2 = good_books[good_books['author2'].isnull() == False].drop(columns=['author1','author3', 'author4', 'author5'])
good_books_author3 = good_books[good_books['author3'].isnull() == False].drop(columns=['author2','author1', 'author4', 'author5'])
good_books_author4 = good_books[good_books['author4'].isnull() == False].drop(columns=['author2','author3', 'author1', 'author5'])
good_books_author5 = good_books[good_books['author5'].isnull() == False].drop(columns=['author2','author3', 'author4', 'author1'])

In [43]:
good_books_author1.rename(columns={'author1': 'author'}, inplace=True)
good_books_author2.rename(columns={'author2': 'author'}, inplace=True)
good_books_author3.rename(columns={'author3': 'author'}, inplace=True)
good_books_author4.rename(columns={'author4': 'author'}, inplace=True)
good_books_author5.rename(columns={'author5': 'author'}, inplace=True)

In [44]:
good_books_concat = pd.concat([good_books_author1, good_books_author2, good_books_author3, good_books_author4, good_books_author5])

In [45]:
good_books_concat.groupby('author',as_index=False)['shared_number_of_reviews_with_a_text'].sum()

,author,shared_number_of_reviews_with_a_text
0,Alan R. Clarke,27890.0
1,B.M. Mooyaart-Doubleday,6942.0
2,Bernard Knox,1620.0
3,CelÃ¢l Ãster,15173.0
4,David J. Skal,5754.0
...,...,...
105,Veronica Roth,156896.0
106,William Golding,26886.0
107,William Goldman,15630.0
108,William Shakespeare,7389.0


### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

In [46]:
good_books_concat.groupby('original_publication_year')[['author','shared_number_of_reviews_with_a_text']].max()

,author,shared_number_of_reviews_with_a_text
original_publication_year,,
-720.0,Homer,1620.0
1595.0,William Shakespeare,7389.0
1811.0,Jane Austen,3842.0
1813.0,Jane Austen,49152.0
1818.0,Mary Wollstonecraft Shelley,6664.0
1847.0,Emily BrontÃ«,15606.0
1859.0,Charles Dickens,4364.0
1868.0,Louisa May Alcott,17090.0
1884.0,Mark Twain,4149.0


### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

In [55]:
common = good_books.merge(books, left_on='isbn', right_on='ISBN', how="inner")
common.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,shared_number_of_reviews_with_a_text,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Book-Rating
0,11,77203,77203,3295919,283,1594480001,9.781594e+12,Khaled Hosseini,2003.0,The Kite Runner,...,59730.0,1594480001,The Kite Runner,Khaled Hosseini,2004,Riverhead Books,http://images.amazon.com/images/P/1594480001.0...,http://images.amazon.com/images/P/1594480001.0...,http://images.amazon.com/images/P/1594480001.0...,2.400000
1,18,5,5,2402163,376,043965548X,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck",1999.0,Harry Potter and the Prisoner of Azkaban,...,12033.0,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,2004,Scholastic Paperbacks,http://images.amazon.com/images/P/043965548X.0...,http://images.amazon.com/images/P/043965548X.0...,http://images.amazon.com/images/P/043965548X.0...,3.533333
2,60,1618,1618,4259809,183,1400032717,9.781400e+12,Mark Haddon,2003.0,The Curious Incident of the Dog in the Night-Time,...,35796.0,1400032717,The Curious Incident of the Dog in the Night-T...,Mark Haddon,2004,Vintage,http://images.amazon.com/images/P/1400032717.0...,http://images.amazon.com/images/P/1400032717.0...,http://images.amazon.com/images/P/1400032717.0...,4.813187
3,75,227443,227443,3185154,193,014028009X,9.780140e+12,Helen Fielding,1996.0,NaN,...,8157.0,014028009X,Bridget Jones's Diary,Helen Fielding,1999,Penguin Books,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,http://images.amazon.com/images/P/014028009X.0...,3.751852
4,90,231804,231804,1426690,156,014038572X,9.780140e+12,S.E. Hinton,1967.0,The Outsiders,...,22662.0,014038572X,The Outsiders (Now in Speak!),S. E. Hinton,1997,Puffin Books,http://images.amazon.com/images/P/014038572X.0...,http://images.amazon.com/images/P/014038572X.0...,http://images.amazon.com/images/P/014038572X.0...,4.460674


In [56]:
common['difference'] = common['average_rating'] - common['average']
common['difference']

0    0.004046
1    0.002134
2   -0.002457
3    0.003502
4   -0.000180
Name: difference, dtype: float64

### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

In [57]:
Age_known = users[users['Age'].isnull() == False]
Age_known

,User-ID,Age,City,Region,Country
1,2,18.0,stockton,california,usa
3,4,17.0,porto,v.n.gaia,portugal
5,6,61.0,santa monica,california,usa
9,10,26.0,albacete,wisconsin,spain
10,11,14.0,melbourne,victoria,australia
...,...,...,...,...,...
278848,278849,23.0,georgetown,ontario,canada
278850,278851,33.0,dallas,texas,usa
278851,278852,32.0,brisbane,queensland,australia
278852,278853,17.0,stranraer,n/a,united kingdom


In [59]:
unknown_age = users[users['Age'].isnull()]
zero = Age_known[Age_known['Age'] <= 14]
my_df =[zero]
for x in range(30):
    
    x = Age_known[(Age_known['Age'] >= x * 10 + 5) & (Age_known['Age'] < (x+1) * 10 + 5)]
    if len(x) > 0:
        my_df.append(x)  

### 11. Find the books that appear only in the goodbooks datasets.

In [30]:
temp = good_books.merge(books, left_on="isbn", right_on="ISBN", how="left")
temp[temp['ISBN'].isnull()== True]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,shared_number_of_reviews_with_a_text,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Book-Rating
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,155254.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,37934.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,95009.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,72586.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,51992.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,5297,5297,1858012,2303,375751513,9.780376e+12,"Oscar Wilde, Jeffrey Eugenides",1891.0,The Picture of Dorian Gray,...,9824.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,96,13536860,13536860,18034963,133,345803507,9.780346e+12,E.L. James,2012.0,Fifty Shades Freed,...,25287.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,97,17245,17245,3165724,2207,393970124,9.780394e+12,"Bram Stoker, Nina Auerbach, David J. Skal",1897.0,Dracula,...,5754.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,98,5060378,5060378,6976108,209,307269981,9.780307e+12,"Stieg Larsson, Reg Keeland",2006.0,Flickan som lekte med elden,...,15712.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [63]:
books.groupby(['Book-Author','Book-Title'], as_index=False).count().sort_values('ISBN', ascending=False).reset_index(drop=True).iloc[0]

Book-Author            Louisa May Alcott
Book-Title                  Little Women
ISBN                                  21
Year-Of-Publication                   21
Publisher                             21
Image-URL-S                           21
Image-URL-M                           21
Image-URL-L                           21
Book-Rating                           21
Name: 0, dtype: object

### 13. Find the author with the highest average rating according to the goodbooks datasets.

In [32]:
good_books_concat.groupby('author')[['author', 'average']].mean().idxmax()

average     Rufus Beck
dtype: object

In [33]:
good_books_concat[good_books_concat['author'] == ' Rufus Beck']

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,average,author,authors_count,shared_number_of_reviews_with_a_text
17,18,5,5,2402163,376,043965548X,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck",1999.0,Harry Potter and the Prisoner of Azkaban,...,20413,166129,509447,1266670,https://images.gr-assets.com/books/1499277281m...,https://images.gr-assets.com/books/1499277281s...,4.527866,Rufus Beck,3,12033.0
